## Cattle GTEX

In [1]:
import polars as pl
from pathlib import Path
from pycomfort.files import *
from pycomfort import files
import pyarrow
import pandas as pd
from functional import seq
from typing import *

In [2]:
import sys

local = (Path("..") / "rna_clock").resolve()
if local.exists():
    sys.path.insert(0, Path("..").absolute().as_posix())
    sys.path.insert(0, local)
    print(sys.path)
    %load_ext autoreload
    %autoreload 2

[PosixPath('/data/sources/rna-clock/rna_clock'), '/data/sources/rna-clock/notebooks/..', '/data/sources/rna-clock/notebooks', '/data/users/antonkulaga/micromamba/envs/rna-clock/lib/python310.zip', '/data/users/antonkulaga/micromamba/envs/rna-clock/lib/python3.10', '/data/users/antonkulaga/micromamba/envs/rna-clock/lib/python3.10/lib-dynload', '', '/data/users/antonkulaga/micromamba/envs/rna-clock/lib/python3.10/site-packages']


In [3]:
from rna_clock.splits import *
from rna_clock import *
from rna_clock.trees import *
from rna_clock.config import *
from rna_clock.preprocess import *

In [4]:
pd.set_option("max_colwidth", None)
pd.set_option("max_seq_items", None)
pl.Config.set_tbl_width_chars(10000)
pl.Config.set_fmt_str_lengths(1000)
pl.Config.set_tbl_rows(20)

polars.cfg.Config

In [5]:
locations = Locations(Path(".."))
tprint(locations.cattle_input)

input
	Metadata_FarmGTEx_cattle_V0.tsv
	Gene_read_counts_FarmGTEx_cattle_V0.tsv
	TPM.8742samples_27607genes.tsv
	.gitignore


In [6]:
cols = pl.scan_csv(locations.cattle_expressions, has_header=True, sep="\t").fetch(1).columns
dtypes = cols_of(cols[1:])

In [6]:
cattle_df = pl.read_csv(locations.cattle_expressions, sep="\t", dtypes=dtypes)
cattle_df.head(10)

NameError: name 'dtypes' is not defined

In [8]:
metadata = pl.read_csv(locations.cattle_input / "Metadata_FarmGTEx_cattle_V0.tsv", sep = "\t")
metadata

Sample,Bioproject,Subspecies,breed class,Tissue_name,Tissue_class,Tissue_categories,Clean data,Clean reads,Mapping rate,LibraryLayout,Sex,Age
str,str,str,str,str,str,str,str,str,f64,str,str,str
"""ERS3175530""","""PRJEB31379""","""Bos indicus × Bos taurus""","""Bos indicus x Bos taurus""","""Muscle""","""Muscle""","""Muscle/Heart""","""5,494,351,213""","""28,468,141""",0.8859,"""PAIRED""","""M""","""12.91±8.69 months"""
"""ERS3175550""","""PRJEB31379""","""Bos indicus × Bos taurus""","""Bos indicus x Bos taurus""","""Muscle""","""Muscle""","""Muscle/Heart""","""4,605,832,704""","""23,988,712""",0.8566,"""PAIRED""","""M""","""12.91±8.69 months"""
"""ERS3175562""","""PRJEB31379""","""Bos indicus × Bos taurus""","""Bos indicus x Bos taurus""","""Muscle""","""Muscle""","""Muscle/Heart""","""5,260,268,654""","""27,255,278""",0.8652,"""PAIRED""","""M""","""12.91±8.69 months"""
"""SRS4621166""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""29,716,625,508""","""102,118,988""",0.6208,"""PAIRED""","""M""","""1 year"""
"""SRS4621163""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""27,336,430,758""","""92,981,057""",0.6339,"""PAIRED""","""M""","""1 year"""
"""SRS4621162""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""19,081,928,370""","""65,799,753""",0.5882,"""PAIRED""","""M""","""1 year"""
"""SRS4621167""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""22,247,325,930""","""76,714,917""",0.747,"""PAIRED""","""M""","""1 year"""
"""SRS4621164""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""25,501,952,430""","""87,937,767""",0.7393,"""PAIRED""","""F""","""1 year"""
"""SRS4621165""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""24,784,691,080""","""85,464,452""",0.7666,"""PAIRED""","""M""","""1 year"""


## Cleaning

In [9]:
has_unknowns = ~pl.col("Age").str.contains("known")
metadata_with_age = metadata.filter(has_unknowns)
counts_with_age = metadata.filter(has_unknowns).groupby(pl.col("Age")).agg([pl.count("Age").alias("count")]).sort(pl.col("count"), True)
counts_with_age

Age,count
str,u32
"""0.5 year""",289
"""25 months""",178
"""1.31 years""",158
"""2 years""",125
"""4~6y""",117
"""1.25 years""",100
"""16-20 months""",86
"""7-12 weeks""",86
"""1 year""",85


In [10]:
counts_all = metadata.groupby(pl.col("Age")).agg([pl.count("Age").alias("count")]).sort(pl.col("count"), True)
(counts_all.select(pl.col("count").sum()), counts_with_age.select(pl.col("count").sum()))

(shape: (1, 1)
 ┌───────┐
 │ count │
 │ ---   │
 │ u32   │
 ╞═══════╡
 │ 8653  │
 └───────┘,
 shape: (1, 1)
 ┌───────┐
 │ count │
 │ ---   │
 │ u32   │
 ╞═══════╡
 │ 2867  │
 └───────┘)

In [11]:
metadata_with_age

Sample,Bioproject,Subspecies,breed class,Tissue_name,Tissue_class,Tissue_categories,Clean data,Clean reads,Mapping rate,LibraryLayout,Sex,Age
str,str,str,str,str,str,str,str,str,f64,str,str,str
"""ERS3175530""","""PRJEB31379""","""Bos indicus × Bos taurus""","""Bos indicus x Bos taurus""","""Muscle""","""Muscle""","""Muscle/Heart""","""5,494,351,213""","""28,468,141""",0.8859,"""PAIRED""","""M""","""12.91±8.69 months"""
"""ERS3175550""","""PRJEB31379""","""Bos indicus × Bos taurus""","""Bos indicus x Bos taurus""","""Muscle""","""Muscle""","""Muscle/Heart""","""4,605,832,704""","""23,988,712""",0.8566,"""PAIRED""","""M""","""12.91±8.69 months"""
"""ERS3175562""","""PRJEB31379""","""Bos indicus × Bos taurus""","""Bos indicus x Bos taurus""","""Muscle""","""Muscle""","""Muscle/Heart""","""5,260,268,654""","""27,255,278""",0.8652,"""PAIRED""","""M""","""12.91±8.69 months"""
"""SRS4621166""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""29,716,625,508""","""102,118,988""",0.6208,"""PAIRED""","""M""","""1 year"""
"""SRS4621163""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""27,336,430,758""","""92,981,057""",0.6339,"""PAIRED""","""M""","""1 year"""
"""SRS4621162""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""19,081,928,370""","""65,799,753""",0.5882,"""PAIRED""","""M""","""1 year"""
"""SRS4621167""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""22,247,325,930""","""76,714,917""",0.747,"""PAIRED""","""M""","""1 year"""
"""SRS4621164""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""25,501,952,430""","""87,937,767""",0.7393,"""PAIRED""","""F""","""1 year"""
"""SRS4621165""","""PRJNA526966""","""Bos taurus""","""Chinese yellow cattle""","""Muscle""","""Muscle""","""Muscle/Heart""","""24,784,691,080""","""85,464,452""",0.7666,"""PAIRED""","""M""","""1 year"""


In [12]:
cattle_df_aged = cattle_df.join(metadata_with_age.select(pl.col("Sample").alias("sample_name")), on = "sample_name")
cattle_df_aged.head(10)

sample_name,ENSBTAG00000000005,ENSBTAG00000000008,ENSBTAG00000000009,ENSBTAG00000000010,ENSBTAG00000000011,ENSBTAG00000000012,ENSBTAG00000000013,ENSBTAG00000000014,ENSBTAG00000000015,ENSBTAG00000000016,ENSBTAG00000000019,ENSBTAG00000000020,ENSBTAG00000000021,ENSBTAG00000000022,ENSBTAG00000000023,ENSBTAG00000000024,ENSBTAG00000000025,ENSBTAG00000000026,ENSBTAG00000000027,ENSBTAG00000000029,ENSBTAG00000000030,ENSBTAG00000000031,ENSBTAG00000000032,ENSBTAG00000000033,ENSBTAG00000000037,ENSBTAG00000000039,ENSBTAG00000000040,ENSBTAG00000000042,ENSBTAG00000000044,ENSBTAG00000000046,ENSBTAG00000000049,ENSBTAG00000000050,ENSBTAG00000000052,ENSBTAG00000000053,ENSBTAG00000000054,ENSBTAG00000000056,...,ENSBTAG00000055280,ENSBTAG00000055281,ENSBTAG00000055282,ENSBTAG00000055283,ENSBTAG00000055284,ENSBTAG00000055285,ENSBTAG00000055286,ENSBTAG00000055287,ENSBTAG00000055288,ENSBTAG00000055289,ENSBTAG00000055290,ENSBTAG00000055291,ENSBTAG00000055292,ENSBTAG00000055293,ENSBTAG00000055294,ENSBTAG00000055295,ENSBTAG00000055296,ENSBTAG00000055297,ENSBTAG00000055298,ENSBTAG00000055299,ENSBTAG00000055300,ENSBTAG00000055301,ENSBTAG00000055302,ENSBTAG00000055303,ENSBTAG00000055304,ENSBTAG00000055305,ENSBTAG00000055306,ENSBTAG00000055307,ENSBTAG00000055308,ENSBTAG00000055309,ENSBTAG00000055310,ENSBTAG00000055311,ENSBTAG00000055312,ENSBTAG00000055313,ENSBTAG00000055314,ENSBTAG00000055315,ENSBTAG00000055316
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,...,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""CRS000193""",0.732601,0.0,3.515941,17.065414,0.0,6.954539,13.212291,124.467857,0.140111,0.202199,81.304085,4.169378,20.610147,16.80105,6.014866,14.314436,77.237411,7.485627,0.0,0.0,2.790164,2.359579,1.816498,0.275373,10.215543,10.236467,4.385624,11.108931,0.0,2.909449,5.322211,45.983025,0.184767,0.558067,0.655353,5.017912,...,0.31163,8.234308,0.0,0.0,0.0,0.0,15.626707,0.0,0.0,0.471758,0.101733,0.0,0.0,0.0,0.0,0.115045,136.359085,0.0,0.0,0.327409,0.0,0.0,0.073349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.077509,0.223423,0.0,0.0,0.066384,22.477654
"""CRS000192""",0.404525,0.0,3.844879,17.465862,0.0,7.648959,14.540008,91.574661,0.085029,0.0,80.366692,9.447894,16.625805,13.854832,7.365909,16.632765,72.445496,7.599295,0.0,0.0,1.931239,1.04152,1.303436,0.162048,11.123465,13.467713,5.806954,6.988097,0.0,4.806313,6.627659,41.186932,0.204707,0.530661,0.784354,4.303435,...,0.0,6.302627,0.0,0.0,0.0,0.0,13.179042,0.0,0.0,0.482736,0.046581,0.0,0.0,1.195609,0.0,0.172617,244.633514,0.0,0.0,0.0,0.053473,0.0,0.098639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025252,0.10872,0.0,0.0,0.0,45.580505
"""CRS000191""",1.129043,0.0,3.279143,14.244446,0.0,8.054182,12.744687,91.849876,0.128653,0.302984,72.996643,4.505445,14.547853,11.257021,5.148545,14.573661,58.366505,7.897287,1.976718,0.0,2.189914,2.321794,2.481693,0.163629,7.487403,9.442446,4.475757,10.267601,0.0,1.995125,5.302141,35.490601,0.418292,1.002264,0.884162,4.694433,...,0.0,3.885783,0.0,0.0,0.0,0.0,8.539361,0.0,0.0,0.643033,0.020134,0.0,0.0,0.0,0.0,0.0,195.347366,0.0,0.0,0.0,0.0,0.0,0.253327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.082131,0.0,0.0,0.0,29.584438
"""CRS000190""",0.057311,0.0,1.499,12.340978,0.0,5.037467,10.566368,125.130516,0.0,0.127177,67.101875,5.847825,12.682891,11.316717,6.611647,16.020241,69.813896,6.549657,0.0,0.0,2.631891,1.693438,0.604431,0.138493,9.85006,9.258244,6.49333,17.527864,0.001776,5.032931,3.823147,36.02689,0.209376,0.663654,0.650752,3.444505,...,0.42633,5.365369,0.0,0.0,0.0,0.0,13.371004,0.0,0.0,0.094349,0.062643,0.087869,0.0,0.0,0.0,0.0,182.689804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045737,0.0,0.0,0.05504,40.731606
"""CRS000189""",0.41104,0.0,1.578625,48.145161,0.0,2.591918,4.594988,381.500366,0.15399,0.094593,34.934963,0.678465,10.154993,9.909562,3.716714,17.399958,32.168098,13

In [13]:
###
#monthly = metadata.filter(pl.col("Age").str.contains("month"))
#print(monthly.shape)
#months_distribution = monthly.select(pl.col("Age").unique())
#print(months_distribution.shape)
#months_distribution
###

In [ ]:
def col_sum(exclude: Union[str, list[str]], final_name: str = "sum"):
    return pl.fold(acc=pl.lit(0.0), f=lambda acc, x: acc + x, exprs=pl.all().exclude(exclude)).alias(final_name)

In [30]:
def binary_with_tpm_row_average(df: pl.DataFrame, exclude: list[str], row_sum: float = 1e6):
    avg = row_sum / (len(df.columns) - len(exclude))
    other_cols = seq(exclude).map(lambda s: pl.col(s)).to_list()
    to_select =other_cols + [pl.exclude(exclude) > avg]
    return df.select(to_select)

In [31]:
binary_with_tpm_row_average(cattle_df_aged, "sample_name")

sample_name,ENSBTAG00000000005,ENSBTAG00000000008,ENSBTAG00000000009,ENSBTAG00000000010,ENSBTAG00000000011,ENSBTAG00000000012,ENSBTAG00000000013,ENSBTAG00000000014,ENSBTAG00000000015,ENSBTAG00000000016,ENSBTAG00000000019,ENSBTAG00000000020,ENSBTAG00000000021,ENSBTAG00000000022,ENSBTAG00000000023,ENSBTAG00000000024,ENSBTAG00000000025,ENSBTAG00000000026,ENSBTAG00000000027,ENSBTAG00000000029,ENSBTAG00000000030,ENSBTAG00000000031,ENSBTAG00000000032,ENSBTAG00000000033,ENSBTAG00000000037,ENSBTAG00000000039,ENSBTAG00000000040,ENSBTAG00000000042,ENSBTAG00000000044,ENSBTAG00000000046,ENSBTAG00000000049,ENSBTAG00000000050,ENSBTAG00000000052,ENSBTAG00000000053,ENSBTAG00000000054,ENSBTAG00000000056,...,ENSBTAG00000055280,ENSBTAG00000055281,ENSBTAG00000055282,ENSBTAG00000055283,ENSBTAG00000055284,ENSBTAG00000055285,ENSBTAG00000055286,ENSBTAG00000055287,ENSBTAG00000055288,ENSBTAG00000055289,ENSBTAG00000055290,ENSBTAG00000055291,ENSBTAG00000055292,ENSBTAG00000055293,ENSBTAG00000055294,ENSBTAG00000055295,ENSBTAG00000055296,ENSBTAG00000055297,ENSBTAG00000055298,ENSBTAG00000055299,ENSBTAG00000055300,ENSBTAG00000055301,ENSBTAG00000055302,ENSBTAG00000055303,ENSBTAG00000055304,ENSBTAG00000055305,ENSBTAG00000055306,ENSBTAG00000055307,ENSBTAG00000055308,ENSBTAG00000055309,ENSBTAG00000055310,ENSBTAG00000055311,ENSBTAG00000055312,ENSBTAG00000055313,ENSBTAG00000055314,ENSBTAG00000055315,ENSBTAG00000055316
str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,...,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
"""CRS000193""",false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
"""CRS000192""",false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true
"""CRS000191""",false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
"""CRS000190""",false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true
"""CRS000189""",false,false,false,true,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,true,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,

In [27]:
df.select(pl.exclude(exclude) > avg)

ENSBTAG00000000005,ENSBTAG00000000008,ENSBTAG00000000009,ENSBTAG00000000010,ENSBTAG00000000011,ENSBTAG00000000012,ENSBTAG00000000013,ENSBTAG00000000014,ENSBTAG00000000015,ENSBTAG00000000016,ENSBTAG00000000019,ENSBTAG00000000020,ENSBTAG00000000021,ENSBTAG00000000022,ENSBTAG00000000023,ENSBTAG00000000024,ENSBTAG00000000025,ENSBTAG00000000026,ENSBTAG00000000027,ENSBTAG00000000029,ENSBTAG00000000030,ENSBTAG00000000031,ENSBTAG00000000032,ENSBTAG00000000033,ENSBTAG00000000037,ENSBTAG00000000039,ENSBTAG00000000040,ENSBTAG00000000042,ENSBTAG00000000044,ENSBTAG00000000046,ENSBTAG00000000049,ENSBTAG00000000050,ENSBTAG00000000052,ENSBTAG00000000053,ENSBTAG00000000054,ENSBTAG00000000056,ENSBTAG00000000057,...,ENSBTAG00000055280,ENSBTAG00000055281,ENSBTAG00000055282,ENSBTAG00000055283,ENSBTAG00000055284,ENSBTAG00000055285,ENSBTAG00000055286,ENSBTAG00000055287,ENSBTAG00000055288,ENSBTAG00000055289,ENSBTAG00000055290,ENSBTAG00000055291,ENSBTAG00000055292,ENSBTAG00000055293,ENSBTAG00000055294,ENSBTAG00000055295,ENSBTAG00000055296,ENSBTAG00000055297,ENSBTAG00000055298,ENSBTAG00000055299,ENSBTAG00000055300,ENSBTAG00000055301,ENSBTAG00000055302,ENSBTAG00000055303,ENSBTAG00000055304,ENSBTAG00000055305,ENSBTAG00000055306,ENSBTAG00000055307,ENSBTAG00000055308,ENSBTAG00000055309,ENSBTAG00000055310,ENSBTAG00000055311,ENSBTAG00000055312,ENSBTAG00000055313,ENSBTAG00000055314,ENSBTAG00000055315,ENSBTAG00000055316
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,...,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true
false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true
false,false,false,true,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,true,false,false,false,false,false,...,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false


In [ ]:
ex = ["average"] + exclude if isinstance(exclude, list) else ["average", exclude]
selected_cols = pl.exclude(ex)

In [ ]:
print("test")